In [21]:
import numpy as np
import pandas as pd
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [22]:
events_dict = {
    'BLUE: first_blood': 0,
    'BLUE: dragon': 1,
    'BLUE: herald': 2,
    'BLUE: first_tower_top': 3,
    'BLUE: first_tower_mid': 4,
    'BLUE: first_tower_bot': 5,
    'BLUE: second_tower_top': 6,
    'BLUE: second_tower_mid': 7,
    'BLUE: second_tower_bot': 8,
    'BLUE: third_tower_top': 9,
    'BLUE: third_tower_mid': 10,
    'BLUE: third_tower_bot': 11,
    'BLUE: inhibitor_top': 12,
    'BLUE: inhibitor_mid': 13,
    'BLUE: inhibitor_bot': 14,
    'BLUE: baron': 15,
    'BLUE: elder_dragon': 16,
    'BLUE: nexus_tower': 17,
    'BLUE: nexus': 18,
    'RED: first_blood': 19,
    'RED: dragon': 20,
    'RED: herald': 21,
    'RED: first_tower_top': 22,
    'RED: first_tower_mid': 23,
    'RED: first_tower_bot': 24,
    'RED: second_tower_top': 25,
    'RED: second_tower_mid': 26,
    'RED: second_tower_bot': 27,
    'RED: third_tower_top': 28,
    'RED: third_tower_mid': 29,
    'RED: third_tower_bot': 30,
    'RED: inhibitor_top': 31,
    'RED: inhibitor_mid': 32,
    'RED: inhibitor_bot': 33,
    'RED: baron': 34,
    'RED: elder_dragon': 35,
    'RED: nexus_tower': 36,
    'RED: nexus': 37
}

df = pd.read_csv('data/clean-one-line.csv')
df = df.drop('game', axis=1)
data = df.values.tolist()

for game in data:
    for i, s in enumerate(game):
        if s in events_dict:
            game[i] = events_dict.get(s)

labelsArray = []

for game in data:
    labelsArray += game

sequence = [b for b in labelsArray if not(isinstance(b, float))]
n_in = 5

In [23]:
def splitSequence(seq, n_steps):    
    #Declare X and y as empty list
    X = []
    
    for i in range(len(seq)):
        #get the last index
        lastIndex = i + n_steps
        
        #if lastIndex is greater than length of sequence then break
        if lastIndex > len(seq) - 1:
            break
            
        #Create input and output sequence
        seq_X = seq[i:lastIndex]
        
        #append seq_X, seq_y in X and y list
        X.append(seq_X)
    
    return X

In [24]:
x_train, x_test = train_test_split(sequence,test_size=0.15)

In [25]:
x_train[:10]

[34, 23, 36, 33, 1, 36, 5, 37, 7, 1]

In [26]:
train = splitSequence(x_train, n_in)
train[:10]

[[34, 23, 36, 33, 1],
 [23, 36, 33, 1, 36],
 [36, 33, 1, 36, 5],
 [33, 1, 36, 5, 37],
 [1, 36, 5, 37, 7],
 [36, 5, 37, 7, 1],
 [5, 37, 7, 1, 24],
 [37, 7, 1, 24, 15],
 [7, 1, 24, 15, 4],
 [1, 24, 15, 4, 13]]

In [27]:
test = splitSequence(x_test, n_in)
test[:10]

[[23, 1, 20, 30, 19],
 [1, 20, 30, 19, 24],
 [20, 30, 19, 24, 12],
 [30, 19, 24, 12, 12],
 [19, 24, 12, 12, 17],
 [24, 12, 12, 17, 37],
 [12, 12, 17, 37, 5],
 [12, 17, 37, 5, 8],
 [17, 37, 5, 8, 20],
 [37, 5, 8, 20, 20]]

In [28]:
# source: https://machinelearningmastery.com/develop-encoder-decoder-model-sequence-sequence-prediction-keras/
 
# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, data):
        X1, X2, y = list(), list(), list()
        data = splitSequence(data, n_in)
        for i in range(len(data)):
                # generate source sequence
                source = data[i]
                # define padded target sequence
                target = source[:n_out]
                target.reverse()
                # create padded input target sequence
                target_in = [0] + target[:-1]
                # encode
                src_encoded = to_categorical([source], num_classes=cardinality)
                tar_encoded = to_categorical([target], num_classes=cardinality)
                tar2_encoded = to_categorical([target_in], num_classes=cardinality)
                # store
                X1.append(src_encoded)
                X2.append(tar2_encoded)
                y.append(tar_encoded)
        X1 = np.squeeze(array(X1), axis=1) 
        X2 = np.squeeze(array(X2), axis=1) 
        y = np.squeeze(array(y), axis=1) 
        return X1, X2, y

def get_dataset_with_index(n_in, n_out, cardinality, data, i):
        X1, X2, y = list(), list(), list()
        data = splitSequence(data, n_in)
        # for i in range(len(data)):
        # generate source sequence
        source = data[i]
        # define padded target sequence
        target = source[:n_out]
        target.reverse()
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        src_encoded = to_categorical([source], num_classes=cardinality)
        tar_encoded = to_categorical([target], num_classes=cardinality)
        tar2_encoded = to_categorical([target_in], num_classes=cardinality)
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
        X1 = np.squeeze(array(X1), axis=1) 
        X2 = np.squeeze(array(X2), axis=1) 
        y = np.squeeze(array(y), axis=1) 
        return X1, X2, y

# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
	# define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model
 
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
	# encode
	state = infenc.predict(source)
	# start of sequence input
	target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		yhat, h, c = infdec.predict([target_seq] + state)
		# store prediction
		output.append(yhat[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_seq = yhat
	return array(output)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]
 
# configure problem
n_features = 38 + 1
n_steps_in = 5
n_steps_out = 3
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, x_train)
print(X1.shape,X2.shape,y.shape)
# train model
train.fit([X1, X2], y, epochs=15)

(6311, 5, 39) (6311, 3, 39) (6311, 3, 39)
Epoch 1/15
198/198 [==============================] - 4s 7ms/step - loss: 2.6460 - accuracy: 0.2739
Epoch 2/15
198/198 [==============================] - 1s 6ms/step - loss: 1.4219 - accuracy: 0.4440
Epoch 3/15
198/198 [==============================] - 1s 7ms/step - loss: 0.9807 - accuracy: 0.6125
Epoch 4/15
198/198 [==============================] - 1s 7ms/step - loss: 0.6008 - accuracy: 0.8019
Epoch 5/15
198/198 [==============================] - 1s 6ms/step - loss: 0.3419 - accuracy: 0.9095
Epoch 6/15
198/198 [==============================] - 1s 7ms/step - loss: 0.1989 - accuracy: 0.9587
Epoch 7/15
198/198 [==============================] - 1s 7ms/step - loss: 0.1195 - accuracy: 0.9811
Epoch 8/15
198/198 [==============================] - 1s 6ms/step - loss: 0.0769 - accuracy: 0.9911
Epoch 9/15
198/198 [==============================] - 1s 6ms/step - loss: 0.0507 - accuracy: 0.9966
Epoch 10/15
198/198 [==============================] - 1s 

In [29]:
total, correct = len(x_test), 0
total

1115

In [31]:
# evaluate LSTM
total, correct = len(test), 0
for i in range(total):
	X1, X2, y = get_dataset_with_index(n_steps_in, n_steps_out, n_features, x_test, i)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float( len(test))*100.0))

1/1 [==============================] - 0s 17ms/step
Accuracy: 97.39%


In [ ]:
# spot check some examples
for _ in range(len(x_validate)):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))